In [1]:
import pandas as pd 
import pymongo 
from flask import jsonify
import json
import requests


conn='mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
url = "http://bikeshare.metro.net/stations/json/"
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
response = requests.get(url, headers = headers)

In [2]:
data = (json.loads(response.text))["features"]
data_list = []
for row in data:
    data_list.append({"start_station": row["properties"]["kioskId"],
                      "station_name": row["properties"]["name"],
                      "Latitude" : row["properties"]["latitude"],
                      "Longitude": row["properties"]["longitude"],
                      "KioskPublicStatus" : row["properties"]["kioskPublicStatus"],
                      "KioskStatus" : row["properties"]["kioskStatus"],
                      "BikesAvailable" : row["properties"]["bikesAvailable"],
                      "DocksAvailable" : row["properties"]["docksAvailable"],
                      "TotalDocks" : row["properties"]["totalDocks"],
                      "CloseTime" : row["properties"]["closeTime"],
                      "OpenTime" : row["properties"]["openTime"],
                      "TimeZone" : row["properties"]["timeZone"],
                      "Address" : row["properties"]["addressStreet"],
                      "City" : row["properties"]["addressCity"],
                      "State" : row["properties"]["addressState"],
                      "ZipCode" : row["properties"]["addressZipCode"]
                       })

In [3]:
df = pd.DataFrame(data_list)
df.head()

,Address,BikesAvailable,City,CloseTime,DocksAvailable,KioskPublicStatus,KioskStatus,Latitude,Longitude,OpenTime,State,TimeZone,TotalDocks,ZipCode,start_station,station_name
0,723 Flower Street,9,Los Angeles,23:58:00,18,Active,FullService,34.04850,-118.25854,00:02:00,CA,Pacific Standard Time,27,90017,3005,7th & Flower
1,729 S Olive Street,9,Los Angeles,23:58:00,15,Active,FullService,34.04554,-118.25667,00:02:00,CA,Pacific Standard Time,24,90014,3006,Olive & 8th
2,557 S 5th Street,3,Los Angeles,23:58:00,20,Active,FullService,34.05048,-118.25459,00:02:00,CA,Pacific Standard Time,23,90071,3007,5th & Grand
3,865 S Figueroa Street,7,Los Angeles,23:58:00,8,Active,FullService,34.04661,-118.26273,00:02:00,CA,Pacific Standard Time,15,90017,3008,Figueroa & 9th
4,401 East 11th Street,13,Los Angeles,23:58:00,9,Active,FullService,34.03705,-118.25487,00:02:00,CA,Pacific Standard Time,22,90015,3010,11th & Maple


In [7]:
week_day="Monday"
db = client.bike_data_db
collection = db.bike_trip.find({"$and":[{"start_station": int(station_name)},{"weekday": str(week_day)}]} )

trips = []
for trip in collection:
    trips.append(trip)
df_users = pd.DataFrame(trips)

In [8]:
df_users.head()

,_id,bike_id,duration,end_day,end_lat,end_lon,end_station,end_time,passholder_type,plan_duration,start_day,start_lat,start_lon,start_station,start_time,trip_id,trip_route_category,weekday
0,5c02ecce16209a0decfeea0e,6263,9,2018-07-02,34.037048,-118.254868,3010,11:16:00,Monthly Pass,30,2018-07-02,34.037048,-118.254868,3010,11:07:00,95088566,Round Trip,Monday
1,5c02ecce16209a0decfeeac1,6658,27,2018-07-02,34.056610,-118.237213,3014,15:26:00,Walk-up,1,2018-07-02,34.037048,-118.254868,3010,14:59:00,95116142,One Way,Monday
2,5c02ecce16209a0decfeebbc,6679,9,2018-07-02,34.035801,-118.233170,3058,19:16:00,Monthly Pass,30,2018-07-02,34.037048,-118.254868,3010,19:07:00,95144778,One Way,Monday
3,5c02ecce16209a0decff0111,11982,28,2018-07-09,34.039982,-118.266403,3047,15:32:00,Monthly Pass,30,2018-07-09,34.037048,-118.254868,3010,15:04:00,96292373,One Way,Monday
4,5c02ecce16209a0decff020f,12327,9,2018-07-09,34.035801,-118.233170,3058,19:14:00,Monthly Pass,30,2018-07-09,34.037048,-118.254868,3010,19:05:00,96321259,One Way,Monday


In [9]:
df_merged = df_users.merge(df, on="start_station", how="inner")

In [10]:
df_merged.head()

,_id,bike_id,duration,end_day,end_lat,end_lon,end_station,end_time,passholder_type,plan_duration,...,KioskPublicStatus,KioskStatus,Latitude,Longitude,OpenTime,State,TimeZone,TotalDocks,ZipCode,station_name
0,5c02ecce16209a0decfeea0e,6263,9,2018-07-02,34.037048,-118.254868,3010,11:16:00,Monthly Pass,30,...,Active,FullService,34.03705,-118.25487,00:02:00,CA,Pacific Standard Time,22,90015,11th & Maple
1,5c02ecce16209a0decfeeac1,6658,27,2018-07-02,34.056610,-118.237213,3014,15:26:00,Walk-up,1,...,Active,FullService,34.03705,-118.25487,00:02:00,CA,Pacific Standard Time,22,90015,11th & Maple
2,5c02ecce16209a0decfeebbc,6679,9,2018-07-02,34.035801,-118.233170,3058,19:16:00,Monthly Pass,30,...,Active,FullService,34.03705,-118.25487,00:02:00,CA,Pacific Standard Time,22,90015,11th & Maple
3,5c02ecce16209a0decff0111,11982,28,2018-07-09,34.039982,-118.266403,3047,15:32:00,Monthly Pass,30,...,Active,FullService,34.03705,-118.25487,00:02:00,CA,Pacific Standard Time,22,90015,11th & Maple
4,5c02ecce16209a0decff020f,12327,9,2018-07-09,34.035801,-118.233170,3058,19:14:00,Monthly Pass,30,...,Active,FullService,34.03705,-118.25487,00:02:00,CA,Pacific Standard Time,22,90015,11th & Maple


In [11]:
from datetime import datetime

In [12]:
for time_sl in df_users["start_time"]:
    time_slices = datetime.strptime(time_sl, "%H:%M:%S").strftime("%H")
    print(time_slices)

11
14
19
15
19
19
16
16
22
12
13
13
16
21
18
18
19
19
15
16
16
16
17
17
13
22
16
19
19
19
17
19


In [13]:
df_merged["time_slices"] = [datetime.strptime(time_sl, "%H:%M:%S").strftime("%H") for time_sl in df_merged["start_time"]]


In [14]:
df_merged["time_slices"].min()


'11'

In [15]:
df_grouped = df_merged.groupby("time_slices")["duration"].sum()

In [16]:
df_grouped = df_grouped.reset_index()

In [17]:
df_grouped.sort_values("time_slices")

,time_slices,duration
0,11,9
1,12,11
2,13,123
3,14,27
4,15,52
5,16,114
6,17,56
7,18,45
8,19,70
9,21,4


In [18]:
list(df_grouped)

['time_slices', 'duration']